In [3]:
import boto3
import json
import pandas as pd

In [129]:
from datetime import datetime

In [14]:
client = boto3.client('sagemaker-runtime')

In [15]:
endpoint_name = "insight-bert-2023-11-13-20-40-49-689"                                      
content_type = "application/list-text"  # The MIME type of the input data in the request body.
accept = "application/json" 

In [120]:
def create_payload(question,text):   
    #question=questions.query(f"tag==\"{tag}\"")['question']     
    #print(f"[\"{question.to_string(index=False)}\" ,\"{text}]\"" )
    #return f"[\"{question.to_string(index=False)}\" ,\"{text}]\""
    #print([f"\"{question}\" ,\"{text}\""])
    input_data=[''+question+'',''+text+'']    
    return input_data

In [122]:
def parse_response(response,score_threshold):    
    answer = json.loads(response['Body'].read().decode())
    #print(answer)
    if answer['score'] > score_threshold:
        return answer['answer']
    else:
        return 'NA'

In [97]:
def call_endpoint(input_data):
    payload =json.dumps(input_data).encode("utf-8")    
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, 
        CustomAttributes="", 
        ContentType=content_type,
        Accept=accept,
        Body=payload
        )
    answer=parse_response(response,.05)
    return answer

In [66]:
s3_input_file_path ='s3://abb8-ml-model/dailydownload/mailContents.json'
s3_output_file_path ='s3://abb8-ml-model/dailydownload/insights.json'
s3_prompt_questions_path='./data/questions.txt'

In [138]:
#Process s3 documents
def process_files():
    docs = pd.read_json(s3_input_file_path)
    docs.insert(3,"insight_dates","")
    docs.insert(4,"summary","")
    questions=pd.read_csv(s3_prompt_questions_path)
    #print(docs.head(1))
    #print(questions.head(1))

In [144]:
def run_insight():
    process_files()
    df = pd.DataFrame(columns=['thread_id','text', 'label'])
    result_df= pd.DataFrame(columns=['id','title', 'description','type','status','documents','pocID','poc','url','actionDate','location','createdDate','updatedDate'])
    for index,row in docs.iterrows():
        id=row["thread_id"]
        title=row["mail_subject"]
        #description=
        #'type','status','documents','pocID','poc','url','actionDate','location','createdDate','updatedDate'
        thread_text=row["mail_body"]
        #print(thread_subject)
        #location='location'    
        #input_data = create_payload('location',thread_text)      
        #create data
        for index,row in questions.iterrows():
            payload = create_payload(row["question"],thread_text) 
            #print(payload)
            match row["tag"]:
                case 'location':
                    location=call_endpoint(payload)
                    #print('loc '+location)
                case 'actiondate':
                    actiondate=call_endpoint(payload)
                    #print('actiondate:' + actiondate)  
                case 'poc':
                    poc=call_endpoint(payload) 
                    #print('poc:' +poc) 
                case 'url':
                    url=call_endpoint(payload)    
                    #print('url:'+url)          
                case _:
                    print( row["tag"] +":no action")    
        list_row = [id,title, 'description','event','','','',poc,url,actiondate,location,datetime.now(),'']      
        result_df.loc[len(result_df)]=list_row
    print(result_df)
    result_df.to_json(s3_output_file_path,orient='records')    

In [145]:
run_insight()

     id               title  description   type status documents pocID  \
0  1001        utility_bill  description  event                          
1  1002  utility_bill march  description  event                          
2  1001        utility_bill  description  event                          

                     poc                                                url  \
0          our customers  If you have any questions don’t hesitate to re...   
1  Robert  Connor Family                                                 NA   
2    DR. Janet Baker DDS                       by clicking the button below   

                           actionDate                   location  \
0                          10/30/2023                      Pepco   
1                                  NA                         NA   
2  Tuesday, December 16 2023  3:30 PM  Tuesday, December 16 2023   

                 createdDate updatedDate  
0 2023-11-14 17:09:54.773560              
1 2023-11-14 17:09:54.92455

In [7]:
s3_client = boto3.client('s3', 'us-east-1')
Prefix='dailydownload'
key='mailContents.json'
bucket_name = "abb8-ml-model"

In [8]:
s3_bucket = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix='dailydownload'   
    )



In [13]:
for content in s3_bucket["Contents"]:
    print(content["Key"])

dailydownload/
dailydownload/insights.json
dailydownload/mailContent.json
dailydownload/mailContents.json
dailydownload/mailcontents.txt
dailydownload/zephyr-7b-alpha.Q5_K_M.gguf


In [63]:
read_from_s3 = s3_client.get_object(
            Key='dailydownload/mailContents.json',            
            Bucket=bucket_name
        )
#print(read_from_s3['Body'].read().decode('UTF-8'))


In [64]:
dataf=json.dumps(read_from_s3['Body'].read().decode('UTF-8'))
dataf=json.loads(dataf)
print(dataf)

[
    {
        "thread_id": "1001",
        "mail_subject": "utility_bill",
        "mail_body": " Rob, We hope that you’re enjoying your electricity utility service.At  Pepco we take pride in making sure that all of our customers enjoy what we provide them.    I did want to quickly mentin that we haven’t received payment from you just yet./    No worries, it isn’t due until 10/30/2023 (one week from today).You can pay by mailing us a check at Pepco Baltimore or online /    If you have any questions don’t hesitate to reply to this email or give us a call at 1-888-456-7894"
    },
    {
        "thread_id": "1002",
        "mail_subject": "utility_bill march",
        "mail_body": "PARENT-TEACHER CONFERENCE REMINDER -springfield elementary school  Date: ___11/25/2023_______________________ To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference scheduled for ________Robert  Connor Family______________on _11/25/2023______

In [54]:
print(dataf)

"[\r\n    {\r\n        \"thread_id\": \"1001\",\r\n        \"mail_subject\": \"utility_bill\",\r\n        \"mail_body\": \" Rob, We hope that you\u2019re enjoying your electricity utility service.At  Pepco we take pride in making sure that all of our customers enjoy what we provide them.    I did want to quickly mentin that we haven\u2019t received payment from you just yet./    No worries, it isn\u2019t due until 10/30/2023 (one week from today).You can pay by mailing us a check at Pepco Baltimore or online /    If you have any questions don\u2019t hesitate to reply to this email or give us a call at 1-888-456-7894\"\r\n    },\r\n    {\r\n        \"thread_id\": \"1002\",\r\n        \"mail_subject\": \"utility_bill march\",\r\n        \"mail_body\": \"PARENT-TEACHER CONFERENCE REMINDER -springfield elementary school  Date: ___11/25/2023_______________________ To the parent(s) of __Isabella__________________________________,Just a reminder that we have a parent-teacher conference schedu

In [65]:
from pandas import json_normalize
df2 = pd.read_json(dataf, orient ='records')

In [66]:
print(df2)

   thread_id        mail_subject  \
0       1001        utility_bill   
1       1002  utility_bill march   
2       1001        utility_bill   

                                           mail_body  
0   Rob, We hope that you’re enjoying your electr...  
1  PARENT-TEACHER CONFERENCE REMINDER -springfiel...  
2  Hello Natasha,Just a friendly reminder about y...  
